In [89]:
import pandas as pd
import json
from datetime import datetime, timedelta

In [90]:
data = pd.read_csv("DE Assessment/data.csv")

In [91]:
def parse_contracts(contacts_str):
    """Parses the JSON string of contracts into a list of dictionaries."""
    if isinstance(contacts_str,str):
        try:
            return json.loads(contacts_str)
        except json.JSONDecodeError:
            return []
    return[]

In [92]:
def calculate_tot_claim_cnt_l180d(claims, days=180):
    """Counts the number of claims made in the last 180 days."""
    cutoff_date = datetime.now() - timedelta(days=days) 
    valid_claims = [
        claim for claim in claims if isinstance(claim, dict) 
        and claim['claim_date'] 
        and datetime.strptime(claim['claim_date'], '%d.%m.%Y') >= cutoff_date
    ]
    return len(valid_claims) if valid_claims else -3  

In [93]:
def calculate_disb_bank_loan_wo_tbc(loans):
    """Calculates the total exposure of loans excluding TBC loans."""
    return sum(
        float(loan.get('loan_summa', 0)) for loan in loans 
        if isinstance(loan, dict) and loan.get('loan_summa') and loan.get('bank') not in ['LIZ', 'LOM', 'MKO', 'SUG', None]
    ) or -1

In [ ]:
def calculate_day_sinlastloan(loans, application_date):
    """Calculates the number of days since the last loan was taken."""
    if not isinstance(loans, list):
        return -1 
    
    valid_loans = [
        loan for loan in loans if isinstance(loan, dict) and loan['contract_date'] 
        and loan.get('loan_summa') is not None 
    ]
    
    if not valid_loans: 
        return -1
    
    last_loan_date = max(valid_loans, key=lambda loan: datetime.strptime(loan['contract_date'], '%d.%m.%Y'))['contract_date']
    days_since_last_loan = (pd.to_datetime(application_date, errors='coerce').tz_localize(None) - datetime.strptime(last_loan_date, '%d.%m.%Y')).days
    
    return days_since_last_loan if days_since_last_loan >= 0 else -3

In [ ]:
def process_and_save_data(file_csv):
    """Reads the dataset, applies calculations, and saves the output."""
    data = pd.read_csv(file_csv)

    data['parsed_contracts'] = data['contracts'].apply(parse_contracts)

    data['tot_claim_cnt_l180d'] = data['parsed_contracts'].apply(calculate_tot_claim_cnt_l180d)
    data['disb_bank_loan_wo_tbc'] = data['parsed_contracts'].apply(calculate_disb_bank_loan_wo_tbc)
    data['day_sinlastloan'] = data.apply(lambda row: calculate_day_sinlastloan(row['parsed_contracts'], row['application_date']), axis=1)


    output_df = data[['id', 'application_date', 'tot_claim_cnt_l180d', 'disb_bank_loan_wo_tbc', 'day_sinlastloan']]
    output_df.to_csv('contract_output.csv', index=False)



In [96]:
if __name__ == "__main__":
    process_and_save_data('DE Assessment/data.csv')

            id                  application_date  tot_claim_cnt_l180d  \
0    2925210.0  2024-02-12 19:22:46.652000+00:00                   -3   
1    2925211.0  2024-02-12 19:24:29.135000+00:00                   -3   
2    2925212.0  2024-02-12 19:24:41.493000+00:00                   -3   
3    2925213.0  2024-02-12 19:24:29.135000+00:00                   -3   
4    2925214.0  2024-02-12 19:24:56.857000+00:00                   -3   
..         ...                               ...                  ...   
995  2926205.0  2024-02-13 06:09:54.210000+00:00                   -3   
996  2926206.0  2024-02-13 06:09:54.306000+00:00                   -3   
997  2926207.0  2024-02-13 06:09:55.661000+00:00                   -3   
998  2926208.0  2024-02-13 06:09:57.024000+00:00                   -3   
999  2926209.0  2024-02-13 06:10:00.403000+00:00                   -3   

     disb_bank_loan_wo_tbc  day_sinlastloan  
0                     -1.0               -1  
1                     -1.0     